In [1]:
import Thymio as th
import time
import costmap_path as cp
import cv2
import numpy as np

#FIXME: not right original image
original_image = cv2.imread('grid.png',0)
costmap, block_size, start, goal, display_image=cp.init(original_image)
path,costmap = cp.update(costmap, block_size,start, goal, display_image,[])
next_goal = path[0]
current_pos = np.array([start[0],start[1],0])
current_speed = np.array([0,0,0])
camera_position = (0,0,0)
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])
print("Open thymio")
thymio = th.Thymio()
await thymio.iniateLock()
print("Thymio opened")
camera_on = True
while True:
    next_goal = path[0]
    print(f'Next goal : {next_goal}')
    time.sleep(1)
    print(f'Current position : {current_pos}')
    thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:3] # Is used for the controller

    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure = thymio.getSpeedL(), 
                                    V_right_measure = thymio.getSpeedR(), 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2])
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

    obstacle,detected = thymio.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        path,costmap = cp.update(costmap, current_pos, goal, display_image)
        
    if thymio.robot_close_waypoint(current_pos[:2], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            break
        
        
    
    
    
    
    
    
    
    
 







Click two points: Start and Goal.
Mouse clicked at: (100, 87)
Mouse clicked at: (218, 179)
Open thymio
Thymio opened
Next goal : (2, 2)
Current position : [2 2 0]
<class 'float'>
Next goal : (3, 2)
Current position : [2 2 0]
<class 'float'>
Next goal : (3, 3)
Current position : [1.81818182 1.81818182 0.        ]
<class 'float'>
Next goal : (3, 4)
Current position : [ 1.16847826  0.86956522 -0.03804348]
<class 'float'>
Next goal : (4, 4)
Current position : [1.36277585 0.31133962 0.93108314]
<class 'float'>
Goal reached


In [2]:
thymio.set_var("motor.left.target", 0)
thymio.set_var("motor.right.target", 0)